In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import sys
import pickle
from itertools import product

import numpy as np
import pandas as pd
import scipy.stats as stats

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns


sns.set_theme(style='whitegrid')
sns.set_theme(style='ticks')



In [ ]:
repo_dir = Path('../..')

In [ ]:
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))
    
from analysis.curve_fitting.src.fitting_functions import LOSS_FUNCTIONS
from analysis.curve_fitting.src.utils import apply_filters, load_yaml, convert_loss_parameters, convert_loss_parameters_batch

from visualization.src.utils import COLOR_PALETTES, set_ticks, save_figs, COLORS
from visualization.src.visualize import plot_reg, plot_reg_bivariate, plot_confidence_intervals


In [ ]:
args = {
    'results_csv': repo_dir / 'results' / 'benchmark_scores.csv',

}

In [ ]:
results_csv = args['results_csv']
df_results = pd.read_csv(results_csv)


## Experiment Configuration

## Apply Data Filters

In [ ]:
df_untrained = df_results[
    (df_results['is_pretrained'] == False) &
    # (df_results['is_random'] == True) &
    (df_results['is_ablation'] == False) &
    (df_results['is_adv'] == False) &
    (df_results['is_ssl'] == False)
]

df_untrained

random_models = df_untrained[df_untrained.is_random].arch.unique()
df_untrained = df_untrained[(df_untrained.is_random) | (df_untrained.arch.isin(random_models) & (df_untrained.dataset=='imagenet') & (df_untrained.samples_per_class==0))]

df_untrained

## Visualize

#### Plotting settings

In [ ]:
linewidth = 3.0
fig_multiplier = 0.7
figsize = (10, 12)
figsize = (fig_multiplier * figsize[0], fig_multiplier * figsize[1])


color_palette_regions = COLOR_PALETTES['regions']
color_palette_samples = COLOR_PALETTES['samples']
color_palette_models = COLOR_PALETTES['models']
color_palette_datasets = COLOR_PALETTES['datasets']

color_palette = color_palette_datasets

colors = [color_palette[0], color_palette[-1]]

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=figsize, dpi=300)

ax = axes
df = df_untrained.copy()
df = df.rename(columns={'dataset':'Dataset', 'arch_family':'Architecture'})
df['sort'] = df.Architecture.map({
    'ResNet': 0,
    'EfficientNet': 1,
    'ConvNeXt': 2,
    'ViT': 3,
})
model_map = {
    f"resnet{i}": f"ResNet{i}" for i in [18, 34, 50, 101, 152]
}
model_map.update({
    f'efficientnet_b{i}': f'EfficientNet-B{i}' for i in [0, 1, 2]
})
model_map.update({
    f'convnext_{i}': f'ConvNeXt-{i[0].upper()}' for i in ['tiny', 'small', 'base', 'large']
})
model_map.update({
    f'deit_{i}': f'ViT-{i[0].upper()}' for i in ['tiny', 'small', 'base', 'large']
})
df['arch'] = df['arch'].map(model_map)
model_sort = {x:i for i, x in enumerate(model_map.values())}
df['model_sort'] = df.arch.map(model_sort)
df.sort_values(['sort', 'model_sort'], inplace=True)

for idx, r in enumerate(['Neural', 'Behavioral']):
    ax = axes[idx]
    
    if r == 'Neural':
        df_plot = df[df['region'] != 'Behavioral']
        df_plot = df_plot.groupby(['arch', 'is_random', 'model_id', 'model_sort', 'sort']).agg({'score':'mean'}).reset_index()
    else:
        df_plot = df[df['region'] == 'Behavioral']
        
    df_plot = df_plot.sort_values(['sort', 'model_sort'])
        


    sns.barplot(data=df_plot, x='arch', y='score', hue='is_random', ax=ax, palette=colors)

    ax.set_xlabel('Model', fontsize=16, fontweight='bold')
    ax.set_ylabel('Alignment Score (S)', fontsize=16, fontweight='bold')
    ax.set_title(f'{r.capitalize()} Alignment', fontsize=16, fontweight='bold')
    ax.legend(loc='center right', title= 'Untrained')
    ax.grid(True)




    # ax.plot(compute, pred_losses)
    ax.spines[['right', 'top']].set_visible(False)
    # ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
    labels = ax.get_xticklabels()
    # for i, label in enumerate(labels):
    #     labels[i] = f"{label.get_text()} \n ({samples_map[label.get_text()]})"

    ax.set_xticks(ax.get_xticks(), labels, rotation=30, ha='right', rotation_mode='anchor')
    
    
    
plt.tight_layout()


figures_dir = '../figures'
fig_name = 'fig11_untrained'
formats = ['pdf', 'png', 'svg']
save_figs(figures_dir, fig_name, formats=formats)